# Data Preprocessing

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn as skl
import matplotlib.pylab as plt

In [2]:
train = pd.read_csv('AdvWorksCusts.csv')
test = pd.read_csv('AW_test.csv')
target_reg= pd.read_csv('AW_AveMonthSpend.csv')
target_cl = pd.read_csv('AW_BikeBuyer.csv')

In [3]:
 pd.DataFrame.drop_duplicates(train, subset=['CustomerID'], keep='first', inplace=True)
 pd.DataFrame.drop_duplicates(target_reg, subset=['CustomerID'], keep='first', inplace=True)
 pd.DataFrame.drop_duplicates(target_cl, subset=['CustomerID'], keep='first', inplace=True)

In [4]:
join=train.merge(target_reg,on='CustomerID').merge(target_cl,on='CustomerID')

In [5]:
join.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16404 entries, 0 to 16403
Data columns (total 25 columns):
CustomerID              16404 non-null int64
Title                   88 non-null object
FirstName               16404 non-null object
MiddleName              9453 non-null object
LastName                16404 non-null object
Suffix                  2 non-null object
AddressLine1            16404 non-null object
AddressLine2            274 non-null object
City                    16404 non-null object
StateProvinceName       16404 non-null object
CountryRegionName       16404 non-null object
PostalCode              16404 non-null object
PhoneNumber             16404 non-null object
BirthDate               16404 non-null object
Education               16404 non-null object
Occupation              16404 non-null object
Gender                  16404 non-null object
MaritalStatus           16404 non-null object
HomeOwnerFlag           16404 non-null int64
NumberCarsOwned         16404

In [6]:
nn_cols=[col for col in join.columns if join[col].count()>=0.7*len(join)]
print(nn_cols)

['CustomerID', 'FirstName', 'LastName', 'AddressLine1', 'City', 'StateProvinceName', 'CountryRegionName', 'PostalCode', 'PhoneNumber', 'BirthDate', 'Education', 'Occupation', 'Gender', 'MaritalStatus', 'HomeOwnerFlag', 'NumberCarsOwned', 'NumberChildrenAtHome', 'TotalChildren', 'YearlyIncome', 'AveMonthSpend', 'BikeBuyer']


In [7]:
join=join[nn_cols]

In [8]:
join.isnull().sum()

CustomerID              0
FirstName               0
LastName                0
AddressLine1            0
City                    0
StateProvinceName       0
CountryRegionName       0
PostalCode              0
PhoneNumber             0
BirthDate               0
Education               0
Occupation              0
Gender                  0
MaritalStatus           0
HomeOwnerFlag           0
NumberCarsOwned         0
NumberChildrenAtHome    0
TotalChildren           0
YearlyIncome            0
AveMonthSpend           0
BikeBuyer               0
dtype: int64

In [9]:
join.dtypes

CustomerID               int64
FirstName               object
LastName                object
AddressLine1            object
City                    object
StateProvinceName       object
CountryRegionName       object
PostalCode              object
PhoneNumber             object
BirthDate               object
Education               object
Occupation              object
Gender                  object
MaritalStatus           object
HomeOwnerFlag            int64
NumberCarsOwned          int64
NumberChildrenAtHome     int64
TotalChildren            int64
YearlyIncome             int64
AveMonthSpend            int64
BikeBuyer                int64
dtype: object

In [10]:
join.nunique()

CustomerID              16404
FirstName                 662
LastName                  363
AddressLine1            11871
City                      269
StateProvinceName          52
CountryRegionName           6
PostalCode                323
PhoneNumber              7949
BirthDate                7896
Education                   5
Occupation                  5
Gender                      2
MaritalStatus               2
HomeOwnerFlag               2
NumberCarsOwned             5
NumberChildrenAtHome        6
TotalChildren               6
YearlyIncome            15465
AveMonthSpend             152
BikeBuyer                   2
dtype: int64

In [11]:
to_drop = ['FirstName','LastName','City','StateProvinceName','AddressLine1','PostalCode','PhoneNumber']

join.drop(to_drop,inplace=True,axis=1)

In [12]:
join.head()

,CustomerID,CountryRegionName,BirthDate,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,AveMonthSpend,BikeBuyer
0,11000,Australia,1966-04-08,Bachelors,Professional,M,M,1,0,0,2,137947,89,0
1,11001,Australia,1965-05-14,Bachelors,Professional,M,S,0,1,3,3,101141,117,1
2,11002,Australia,1965-08-12,Bachelors,Professional,M,M,1,1,3,3,91945,123,0
3,11003,Australia,1968-02-15,Bachelors,Professional,F,S,0,1,0,0,86688,50,0
4,11004,Australia,1968-08-08,Bachelors,Professional,F,S,1,4,5,5,92771,95,1


In [13]:
join['BirthYear']=pd.to_datetime(join['BirthDate']).dt.year;
join['Age']=1998-join['BirthYear']
join.drop(['BirthDate','BirthYear'],axis=1,inplace=True)

In [14]:
test['BirthYear']=pd.to_datetime(test['BirthDate']).dt.year;
test['Age']=1998-test['BirthYear']
test.drop(['BirthDate','BirthYear'],axis=1,inplace=True)

In [15]:
join.isnull().sum()

CustomerID              0
CountryRegionName       0
Education               0
Occupation              0
Gender                  0
MaritalStatus           0
HomeOwnerFlag           0
NumberCarsOwned         0
NumberChildrenAtHome    0
TotalChildren           0
YearlyIncome            0
AveMonthSpend           0
BikeBuyer               0
Age                     0
dtype: int64

In [16]:
join.nunique()

CustomerID              16404
CountryRegionName           6
Education                   5
Occupation                  5
Gender                      2
MaritalStatus               2
HomeOwnerFlag               2
NumberCarsOwned             5
NumberChildrenAtHome        6
TotalChildren               6
YearlyIncome            15465
AveMonthSpend             152
BikeBuyer                   2
Age                        70
dtype: int64

In [17]:
cat_col=[col for col in join.columns if join[col].nunique()<7]
num_col=list(set([col for col in join.columns if join[col].nunique()>7])-set(['CustomerID']))

In [18]:
print('Categorical features are:',cat_col)
print('')
print('Numerical features are:',num_col)

Categorical features are: ['CountryRegionName', 'Education', 'Occupation', 'Gender', 'MaritalStatus', 'HomeOwnerFlag', 'NumberCarsOwned', 'NumberChildrenAtHome', 'TotalChildren', 'BikeBuyer']

Numerical features are: ['AveMonthSpend', 'Age', 'YearlyIncome']


## Initial EDA

# Regression

In [48]:
join.columns

Index(['CustomerID', 'CountryRegionName', 'Education', 'Occupation', 'Gender',
       'MaritalStatus', 'HomeOwnerFlag', 'NumberCarsOwned',
       'NumberChildrenAtHome', 'TotalChildren', 'YearlyIncome',
       'AveMonthSpend', 'BikeBuyer', 'Age'],
      dtype='object')

In [50]:
cat_cols=cat_col.remove('BikeBuyer')

In [51]:
join_x=pd.get_dummies(join.drop('BikeBuyer',axis=1), columns=cat_col,prefix=cat_col)

In [52]:
X=join_x.drop('AveMonthSpend',axis=1)
Y=join['AveMonthSpend']

Y=Y.ravel().astype('float64')

split_test_size=0.3

from sklearn.model_selection import train_test_split,StratifiedShuffleSplit
Xtrain, Xtest, Ytrain, Ytest= train_test_split(X,Y, test_size=split_test_size, random_state=5)

In [53]:
from catboost import CatBoostRegressor
cb=CatBoostRegressor(iterations=1000,depth=5,eval_metric='RMSE',
                    random_seed=10,learning_rate=.1,verbose=False)

In [54]:
cb.fit(Xtrain,Ytrain,use_best_model=True,eval_set=(Xtest,Ytest),early_stopping_rounds=50,verbose=50)

0:	learn: 69.8234130	test: 69.9817506	best: 69.9817506 (0)	total: 74.5ms	remaining: 1m 14s
50:	learn: 3.7249685	test: 3.7100230	best: 3.7100230 (50)	total: 4.26s	remaining: 1m 19s
100:	learn: 3.3533412	test: 3.3374653	best: 3.3374653 (100)	total: 7.69s	remaining: 1m 8s
150:	learn: 3.2387810	test: 3.2370252	best: 3.2370191 (149)	total: 10.9s	remaining: 1m 1s
200:	learn: 3.1754919	test: 3.1869222	best: 3.1869222 (200)	total: 14s	remaining: 55.7s
250:	learn: 3.1342968	test: 3.1596247	best: 3.1595775 (248)	total: 17.5s	remaining: 52.2s
300:	learn: 3.1092185	test: 3.1450619	best: 3.1450619 (300)	total: 20.5s	remaining: 47.6s
350:	learn: 3.0895728	test: 3.1342118	best: 3.1342118 (350)	total: 24.2s	remaining: 44.7s
400:	learn: 3.0748081	test: 3.1246556	best: 3.1246556 (400)	total: 28.9s	remaining: 43.2s
450:	learn: 3.0576132	test: 3.1127968	best: 3.1127968 (450)	total: 33.2s	remaining: 40.4s
500:	learn: 3.0487726	test: 3.1094669	best: 3.1093054 (497)	total: 36.9s	remaining: 36.7s
550:	learn: 

In [71]:
features=[col for col in join.drop(['BikeBuyer','AveMonthSpend'],axis=1)]
tests=test[features]

In [74]:
test_x=pd.get_dummies(tests, columns=cat_col,prefix=cat_col)

In [75]:
np.savetxt('reg.csv',cb.predict(test_x),delimiter=',')

In [70]:
Xtrain.columns.shape

(42,)